참고문헌:  
\[1\] [16-01 트랜스포머(Transformer)](https://wikidocs.net/31379)


## 인코더의 셀프 어텐션

### 1) 셀프 어텐션의 의미와 이점
어텐션 함수는 주어진 `Query`에 대해서 모든 `Key`와의 유사도를 각각 구합니다.<br>
그리고 구해낸 이 유사도를 가중치로 하여 키와 맵핑되어있는 각각의 `값(Value)`에<br>
에 반영해줍니다. 그리고 유사도가 반영된 `Value`을 모두 가중합하여 리턴합니다. <br>

![](../images/atten_query.png)

어텐션 중에서는 `self-attention`이라는 것이 있습니다. 어텐션을 자기 자신에게 수행한다는 <br>
의미입니다. 앞서 배운 seq2seq에서 어텐션을 사용할 경우의 Q,K,V의 정의를 다시 생각해봅시다.  

``` txt
Q = Query : t 시점의 디코더 셀에서의 은닉 상태
K = keys : 모든 시점의 인코더 셀의 은닉 상태들
V = Values : 모든 시점의 인코더 셀의 은닉 상태들
```

사실 t 시점이라는 것은 계속 변화하면서 반복적으로 쿼리를 수행하므로 <br>
결국 전체 시점에 대해서 일반화를 할 수도 있음.  

``` txt
Q = Query : 모든 시점의 디코더 셀에서의 은닉 상태
K = keys : 모든 시점의 인코더 셀의 은닉 상태들
V = Values : 모든 시점의 인코더 셀의 은닉 상태들
```

이처럼 기존에는 디코더 셀의 은닉 상태가 Q이고 인코더 셀의 은닉 상태가 K라는 점에서 <br>
Q와 K가 서로 다른 값을 가지고 있었습니다. 그런데 셀프 어텐션에서는 Q,K,V가 전부 동일 <br>
합니다. 트랜스포머의 셀프 어텐션에서의 Q,K,V는 아래와 같습니다.  <br>

``` txt
Q : 입력 문장의 모든 벡터들
K : 입력 문장의 모든 단어 벡터들  
V : 입력 문장의 모든 단어 벡터들  
```

셀프 어텐션에 대한 구체적인 사항을 배우기 전에 셀프 어텐션을 통해 얻을 수 있는 <br>
대표적인 효과에 대해서 이해해봅시다. 

![](../images/attention_query2.png)

위의 그림은 트랜스포머에 대한 구글 AI 블로그 포스트에서 가져왔습니다. <br>
위의 예시 문장을 번역하면 <br>
'그 동물은 길을 건너지 않았다. 왜냐하면 그것은 너무 핀곤하였기 때문이다.`라는 의미가 <br>
그런데 여기서 그것(it)에 해당하는 것은 관연 길(stree)일까요? 동물(animal)일까요? <br> 
우리는 피곤한 주체가 동물이라는 것을 아주 쉽게 알 수 있지만 기계는 그렇지 않습니다.<br>
하지만 셀프 어텐션은 입력 문장 내의 단어들끼리 유사도를 구하므로 그것(it)이 동물(animal)과 <br>
연관되었을 확률이 높다는 것을 찾아냅니다. <br>


### 2) Q, K, V 벡터 얻기

앞서 셀프 어텐션은 입력 문장의 단어 벡터들을 가지고 수행한다고 하였는데, 사실 셀프 어텐션은<br>
인코더의 초기 입력인 $d\_model$의 차원을 가지는 단어 벡터들을 사용하여 셀프 어텐션을 수행하는 <br>
것이 아니라 우선 각 단어 벡터들로부터 Q벡터, K벡터, V벡터를 얻는 작업을 거칩니다. 이때 이 Q벡터, <br>
K벡터, V벡터들은 초기 입력인 $d_{model}$의 차원을 가지는 단어 벡터들보다 더 작은 차원을 기지는데, <br>
논문에서는 $d_{model}=512$의 차원을 가졌던 각 단어 벡터들을 `64`의 차원을 가지는 Q벡터, K벡터, <br>
V벡터로 변환하였습니다. <br> 
$d_{model}=512$ 차원의 embedding 벡터값이 `64`차원의 Q,K,V 벡터로 변환됨.  

`64`라는 값은 트랜스포머의 또 다른 하이퍼파라미터인 `num_heads`로 인해 결정되는데, 트랜스포머는<br>
 $d_{model}$을 `num_heads`를 8로 하였음. 그림을 통해 이해해봅시다. 예들 들어 여기서 사용하고 있는<br>
 예문 중 student라는 단어 벡터를 Q, K, V의 벡터로 변환하는 과정을 보겠습니다.

 ![](../images/make_qvk.png)
 
기존의 벡터로부터 더 작은 벡터는 가중치 행렬을 곱하므로서 완성됩니다. 각 가중치 행렬은<br>
$d_{model} \times (d_{model}/ \text{num\_heads})$의 크기를 가집니다. 이 가중치 행렬은  훈련과정에서 <br> 학습됩니다. 즉 논문과 같이 $d_{model}=512$이고 $num\_heads=8$이 라면, 각 벡터에 <br>3개의 서로 다른 가중치 행렬을 곱하고 64의 크기를 가지는 Q,K,V 벡터를 얻어냅니다. 위의 그림은 단어 <br> 벡터(임베딩) 중 student 임베딩 벡터로부터 Q,K,V 벡터를 얻어내는 모습을 보여 줍니다. 모든 단어의 <br> 임베딩 벡터에 위와 같은 과정을 거치면 I, am, a, student는 각각의 Q,K,V 벡터를 얻습니다. <br>

$$ 
\begin{align*}
Q_{student}=W^Q \times student \\
K_{student}=W^k \times student \\
V_{student}=W^V \times student
\end{align*} 
$$ 


### 3) Scaled dot-product Attention
Q, K, V 벡터를 얻었다면 지금부터는 기존에 배운 어텐션 메커니즘과 동일합니다. 각 Q벡터는 모든 K벡터<br>
에 대해서 어텐션 스코어를 구하고, 이를 사용하여 모든 V벡터를 가중합하여 어텐션 값 또는 컨텍스트 벡터를<br>
구하게 됩니다. 그리고 이를 모든 Q벡터에 대해서 반복합니다. <br><br>
트랜스포머는 내적만을 사용하는 어텐션 함수 $score(q,k)=q \cdot k$가 아니라 여기에 특정값으로 나눠준 <br>
어텐션 함수인 $score(q,k)=q \cdot k / \sqrt{n}$를 사용합니다. 이러한 함수를 사용하는 어텐션을 <br>
Scaled dot-product Attention 이라고 합니다. (여기서 $n=d_{k}=64$) <br>

![](../images/scaled_dot_atten_scores.png)  

우선 단어 I에 대한 Q벡터를 기준으로 설명해보겠습니다. 지금부터 설명하는 과정은 am에 대한 Q벡터, a에 대한<br> Q벡터, student에 대한 Q벡터에 대해서도 모두 동일한 과정을 거칩니다. 위의 그림은 단어 I에 대한 Q벡터가 모든<br> K벡터에 대해서 어텐션 스코어를 구하는 것을 보여줍니다. 위의 128과 32는 저자가 임의로 가정한 수치로<br> 신경쓰지 않아도 좋습니다.<br>

위의 그림에서 어텐션 스코어는 각각 단어 I가 단어 I, am, a, student와 얼마나 연관되어 있는지를 보여주<br>
는 수치입니다. 트랜스포머에서는 두 벡터의 내적값을 스케일링하는 값으로 K벡터의 차원을 나타내는 $d_k$에 <br> 루트 씌운 $\sqrt{d_k}$ 사용하는 것을 택했습니다. 앞서 언급하였듯이 논문에서$d_k$ 는 <br>
$d_{model}/ num\_heads$라는 식에 따라서 64의 값을 가지므로 $\sqrt{d_k}$는 8의 값을 가집니다. <br>

![](../images/attention_distribution.png)

이제 어텐션 스코어에 소프트맥스 함수를 사용하여 어텐션 분포(Attention Distribution)을 구하고, 각 V벡터와<br> 가중합하여 어텐션 값(Attention Value)을 구합니다. 이를 단어 I에 대한 어텐션 값 또는 단어 I에 대한<br> 컨텍스트 벡터(context vector)라고도 할 수 있습니다. am에 대한 Q벡터, a에 대 Q벡터, student에 대한 <br>
Q벡터에 대해서도 모두 동일한 과정을 반복하여 각각에 대한 어텐션 값을 구합니다. 그런데 한 가지 의문이 <br>
남습니다. 굳이 이렇게 각 Q벡터마다 일일히 따로 연산할 필요가 있을까요?